In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("./pre_processed_train.csv",index_col="id")
test = pd.read_csv("./pre_processed_test.csv",index_col="id")

X=train.copy()
X_test=test.copy()


y=X.label
y_test=X_test.label

X=X.drop(['label'],axis=1)
X_test=X_test.drop(['label'],axis=1)

train.head()

,label,confidence,social_timestamp,social_karma,syntax_ari,lex_liwc_WC,lex_liwc_Analytic,lex_liwc_Clout,lex_liwc_Authentic,lex_liwc_Tone,...,Tokenized_Segment_1601,Tokenized_Segment_1602,Tokenized_Segment_1603,Tokenized_Segment_1604,Tokenized_Segment_1605,Tokenized_Segment_1606,Tokenized_Segment_1607,Tokenized_Segment_1608,Tokenized_Segment_1609,Tokenized_Segment_1610
id,,,,,,,,,,,,,,,,,,,,,
33181,1,0.8,1521614353,5,1.806818,116,72.64,15.04,89.26,1.00,...,0,0,0,0,0,0,0,0,0,0
2606,0,1.0,1527009817,4,9.429737,109,79.08,76.85,56.75,98.18,...,0,0,0,0,0,0,0,0,0,0
38816,1,0.8,1535935605,2,7.769821,167,33.80,76.38,86.24,25.77,...,0,0,0,0,0,0,0,0,0,0
239,1,0.6,1516429555,0,2.667798,273,2.98,15.25,95.42,79.26,...,0,0,0,0,0,0,0,0,0,0
1421,1,0.8,1539809005,24,7.554238,89,32.22,28.71,84.01,1.00,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# evaluate a given model using cross-validation
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
def evaluate_model(model, X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=42)
    model.fit(X_train, y_train)
    
    y_hat = model.predict(X_valid)
    
    return f1_score(y_valid,y_hat,average="binary")


In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)


In [13]:
import optuna
from sklearn.model_selection import cross_val_score
def lgbm_objective(trial):
    
    num_leaves = trial.suggest_int("num_leaves", 24, 80)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True)
    feature_fraction=trial.suggest_float("feature_fraction", 0.1, 0.9)
    bagging_fraction=trial.suggest_float("bagging_fraction", 0.8, 1)
    max_depth=trial.suggest_int("max_depth",5,30)
    max_bin=trial.suggest_int("max_bin",20,90)
    min_sum_hessian_in_leaf=trial.suggest_int("min_sum_hessian_in_leaf",0,100)
    subsample = trial.suggest_float("subsample", 0.5, 1.0, step=0.1)
    model = LGBMClassifier(
                        application='binary',
                        metric='auc',
                        num_leaves=num_leaves,
                        learning_rate=learning_rate,
                        bagging_fraction=bagging_fraction,
                        max_depth=max_depth,
                        max_bin=max_bin,
                        min_sum_hessian_in_leaf=min_sum_hessian_in_leaf,
                        subsample=subsample,
                        n_jobs=-1
                    )  
    return evaluate_model(model, X, y)

study_lgbm = optuna.create_study(study_name='lgbm_optimze',direction='maximize')
study_lgbm.optimize(lgbm_objective, n_trials=100)

[I 2021-08-29 15:13:21,382] A new study created in memory with name: lgbm_optimze


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=90, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=90
[LightGBM] [Warning] bagging_fraction is set=0.828390233567468, subsample=0.5 will be ignored. Current value: bagging_fraction=0.828390233567468


[I 2021-08-29 15:13:22,191] Trial 0 finished with value: 0.7397260273972602 and parameters: {'num_leaves': 32, 'learning_rate': 0.0063418435156973, 'feature_fraction': 0.22671905863281153, 'bagging_fraction': 0.828390233567468, 'max_depth': 29, 'max_bin': 39, 'min_sum_hessian_in_leaf': 90, 'subsample': 0.5}. Best is trial 0 with value: 0.7397260273972602.


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=65, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=65
[LightGBM] [Warning] bagging_fraction is set=0.8254301742304769, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8254301742304769


[I 2021-08-29 15:13:23,453] Trial 1 finished with value: 0.6987399770904925 and parameters: {'num_leaves': 80, 'learning_rate': 2.522663178763523e-05, 'feature_fraction': 0.8161772758830638, 'bagging_fraction': 0.8254301742304769, 'max_depth': 23, 'max_bin': 82, 'min_sum_hessian_in_leaf': 65, 'subsample': 1.0}. Best is trial 0 with value: 0.7397260273972602.


...

[I 2021-08-29 15:15:22,984] Trial 97 finished with value: 0.7714748784440842 and parameters: {'num_leaves': 72, 'learning_rate': 0.045568472079831765, 'feature_fraction': 0.2320540597628114, 'bagging_fraction': 0.9541174699300674, 'max_depth': 8, 'max_bin': 86, 'min_sum_hessian_in_leaf': 44, 'subsample': 0.7}. Best is trial 55 with value: 0.7883683360258482.


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=38, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=38
[LightGBM] [Warning] bagging_fraction is set=0.9207465287376824, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9207465287376824


[I 2021-08-29 15:15:24,329] Trial 98 finished with value: 0.7637540453074433 and parameters: {'num_leaves': 61, 'learning_rate': 0.018463920119779466, 'feature_fraction': 0.7879469896063788, 'bagging_fraction': 0.9207465287376824, 'max_depth': 7, 'max_bin': 84, 'min_sum_hessian_in_leaf': 38, 'subsample': 0.8}. Best is trial 55 with value: 0.7883683360258482.


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=33, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=33
[LightGBM] [Warning] bagging_fraction is set=0.9656433609692393, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9656433609692393


[I 2021-08-29 15:15:25,467] Trial 99 finished with value: 0.7585089141004862 and parameters: {'num_leaves': 56, 'learning_rate': 0.027287231156819573, 'feature_fraction': 0.7470409758859329, 'bagging_fraction': 0.9656433609692393, 'max_depth': 12, 'max_bin': 80, 'min_sum_hessian_in_leaf': 33, 'subsample': 0.8}. Best is trial 55 with value: 0.7883683360258482.


In [14]:
print(study_lgbm.best_trial)
print(study_lgbm.best_trial.params)

FrozenTrial(number=55, values=[0.7883683360258482], datetime_start=datetime.datetime(2021, 8, 29, 15, 14, 32, 881068), datetime_complete=datetime.datetime(2021, 8, 29, 15, 14, 33, 776981), params={'num_leaves': 61, 'learning_rate': 0.09018623372462836, 'feature_fraction': 0.5345789610841638, 'bagging_fraction': 0.9913907896428181, 'max_depth': 5, 'max_bin': 58, 'min_sum_hessian_in_leaf': 39, 'subsample': 0.8}, distributions={'num_leaves': IntUniformDistribution(high=80, low=24, step=1), 'learning_rate': LogUniformDistribution(high=0.1, low=1e-06), 'feature_fraction': UniformDistribution(high=0.9, low=0.1), 'bagging_fraction': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=30, low=5, step=1), 'max_bin': IntUniformDistribution(high=90, low=20, step=1), 'min_sum_hessian_in_leaf': IntUniformDistribution(high=100, low=0, step=1), 'subsample': DiscreteUniformDistribution(high=1.0, low=0.5, q=0.1)}, user_attrs={}, system_attrs={}, intermediate_values={}, tria